In [22]:
from playwright.async_api import async_playwright
import json
import time
import asyncio

In [24]:
async def scrape_jobs():
    async with async_playwright() as playwright: 
        browser = playwright.chromium.launch(headless=False)
        page = browser.new_page()

        all_jobs = []
        pageCount = 1

        while (pageCount < 2) :
            page.goto(f"https://www.google.com/about/careers/applications/jobs/results?location=India&page={str(pageCount)}")
            page.wait_for_selector("main")
            time.sleep(2)

            jobs = []

            job_list_items = page.locator("ul > li")

            for job_card in job_list_items.element_handles():
                # Only process if there is a title
                title_elem = job_card.query_selector("h3")
                link_elem = job_card.query_selector("a[aria-label*='Learn more']")

                if not title_elem or not link_elem:
                    continue  # skip non-job <li>

                job = {}
                job['Title'] = title_elem.inner_text().strip()
                job['URL'] = 'https://www.google.com/about/careers/applications/' + link_elem.get_attribute("href").strip()

                # Company
                company_elem = job_card.query_selector("span.RP7SMd > span")
                job['Company'] = company_elem.inner_text().strip() if company_elem else ""

                # Location
                location_elem = job_card.query_selector("span.pwO9Dc > span")
                job['Location'] = location_elem.inner_text().strip() if location_elem else ""

                # Experience level
                level_elem = job_card.query_selector("span.wVSTAb")
                job['Level'] = level_elem.inner_text().strip() if level_elem else ""

                # Description
                desc_elem = job_card.query_selector("div.Xsxa1e")
                job['Description'] = desc_elem.inner_text().strip() if desc_elem else ""

                jobs.append(job)
            
            pageCount += 1
            if jobs == []:
                break
            
            all_jobs.extend(jobs)

        browser.close()